In [1]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

data = pd.read_parquet(TRANSFORMED_DATA_DIR / 'yellow_tripdata_features_target.parquet')

In [2]:
data

,rides_previous_672,rides_previous_671,rides_previous_670,rides_previous_669,rides_previous_668,rides_previous_667,rides_previous_666,rides_previous_665,rides_previous_664,rides_previous_663,rides_previous_662,rides_previous_661,rides_previous_660,rides_previous_659,rides_previous_658,rides_previous_657,rides_previous_656,rides_previous_655,rides_previous_654,rides_previous_653,rides_previous_652,rides_previous_651,rides_previous_650,rides_previous_649,rides_previous_648,rides_previous_647,rides_previous_646,rides_previous_645,rides_previous_644,rides_previous_643,rides_previous_642,rides_previous_641,rides_previous_640,rides_previous_639,rides_previous_638,rides_previous_637,rides_previous_636,rides_previous_635,rides_previous_634,rides_previous_633,...,rides_previous_37,rides_previous_36,rides_previous_35,rides_previous_34,rides_previous_33,rides_previous_32,rides_previous_31,rides_previous_30,rides_previous_29,rides_previous_28,rides_previous_27,rides_previous_26,rides_previous_25,rides_previous_24,rides_previous_23,rides_previous_22,rides_previous_21,rides_previous_20,rides_previous_19,rides_previous_18,rides_previous_17,rides_previous_16,rides_previous_15,rides_previous_14,rides_previous_13,rides_previous_12,rides_previous_11,rides_previous_10,rides_previous_9,rides_previous_8,rides_previous_7,rides_previous_6,rides_previous_5,rides_previous_4,rides_previous_3,rides_previous_2,rides_previous_1,pickup_hour,PULocationID,target_rides_next_hour
0,49.0,94.0,84.0,85.0,43.0,22.0,14.0,24.0,19.0,9.0,18.0,18.0,9.0,11.0,10.0,6.0,7.0,12.0,7.0,12.0,7.0,7.0,6.0,10.0,8.0,3.0,0.0,6.0,0.0,3.0,11.0,16.0,26.0,23.0,18.0,14.0,15.0,14.0,9.0,8.0,...,20.0,16.0,12.0,14.0,15.0,12.0,22.0,21.0,14.0,17.0,8.0,8.0,19.0,6.0,8.0,4.0,0.0,5.0,2.0,6.0,16.0,24.0,31.0,13.0,20.0,12.0,9.0,17.0,11.0,7.0,4.0,14.0,19.0,22.0,10.0,25.0,19.0,2019-01-29,4,7.0
1,8.0,3.0,0.0,6.0,0.0,3.0,11.0,16.0,26.0,23.0,18.0,14.0,15.0,14.0,9.0,8.0,17.0,15.0,22.0,8.0,7.0,9.0,20.0,11.0,10.0,5.0,1.0,1.0,4.0,4.0,7.0,24.0,32.0,31.0,26.0,23.0,12.0,13.0,10.0,21.0,...,20.0,12.0,9.0,17.0,11.0,7.0,4.0,14.0,19.0,22.0,10.0,25.0,19.0,14.0,7.0,3.0,1.0,4.0,5.0,5.0,22.0,32.0,37.0,26.0,17.0,18.0,20.0,11.0,19.0,10.0,19.0,15.0,13.0,18.0,21.0,19.0,11.0,2019-01-30,4,15.0
2,10.0,5.0,1.0,1.0,4.0,4.0,7.0,24.0,32.0,31.0,26.0,23.0,12.0,13.0,10.0,21.0,10.0,17.0,22.0,9.0,18.0,22.0,17.0,19.0,5.0,7.0,4.0,2.0,3.0,2.0,13.0,18.0,32.0,27.0,17.0,22.0,20.0,20.0,18.0,16.0,...,17.0,18.0,20.0,11.0,19.0,10.0,19.0,15.0,13.0,18.0,21.0,19.0,11.0,10.0,15.0,2.0,4.0,2.0,3.0,8.0,21.0,27.0,35.0,16.0,20.0,10.0,9.0,17.0,15.0,11.0,23.0,20.0,25.0,19.0,26.0,26.0,17.0,2019-01-31,4,19.0
3,5.0,7.0,4.0,2.0,3.0,2.0,13.0,18.0,32.0,27.0,17.0,22.0,20.0,20.0,18.0,16.0,11.0,14.0,14.0,18.0,30.0,33.0,43.0,38.0,70.0,68.0,39.0,33.0,18.0,5.0,8.0,16.0,18.0,23.0,30.0,23.0,20.0,17.0,15.0,24.0,...,20.0,10.0,9.0,17.0,15.0,11.0,23.0,20.0,25.0,19.0,26.0,26.0,17.0,14.0,19.0,5.0,6.0,2.0,5.0,7.0,18.0,40.0,32.0,16.0,16.0,13.0,16.0,9.0,11.0,14.0,16.0,18.0,22.0,22.0,26.0,28.0,40.0,2019-02-01,4,20.0
4,70.0,68.0,39.0,33.0,18.0,5.0,8.0,16.0,18.0,23.0,30.0,23.0,20.0,17.0,15.0,24.0,22.0,22.0,22.0,26.0,28.0,52.0,38.0,79.0,57.0,66.0,76.0,30.0,28.0,6.0,4.0,2.0,9.0,11.0,21.0,28.0,18.0,21.0,21.0,16.0,...,16.0,13.0,16.0,9.0,11.0,14.0,16.0,18.0,22.0,22.0,26.0,28.0,40.0,18.0,20.0,11.0,5.0,9.0,2.0,15.0,26.0,39.0,37.0,23.0,26.0,18.0,19.0,21.0,21.0,22.0,28.0,24.0,28.0,42.0,43.0,30.0,68.0,2019-02-02,4,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [2]:
from lightgbm import LGBMRegressor

from src.training import train_test_split, get_cutoff_training_date

X_train, y_train, X_test, y_test = train_test_split(data, cutoff_date = get_cutoff_training_date(data), target_column_name='target_rides_next_hour')

In [3]:
# print shapes
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((233112, 674), (233112,), (48840, 674), (48840,))

In [4]:
past_rides_features = [col for col in data.columns if 'rides_previous_' in col]

In [5]:
X_train_past_rides = X_train[past_rides_features]

In [7]:
# instantiate model
model = LGBMRegressor()

# fit model
model.fit(X_train_past_rides, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.654639 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 170094
[LightGBM] [Info] Number of data points in the train set: 233112, number of used features: 672
[LightGBM] [Info] Start training from score 8.917692


LGBMRegressor()

In [13]:
# get predictions
y_pred = model.predict(X_test[past_rides_features])

from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(y_test, y_pred)
print(f'RMSE: {rmse=:.4f}')

RMSE: rmse=7.7553
